In [147]:
import pandas as pd
import numpy as np


1、最终目的获取终端用户的信息描述：如猜测对方性别比重、年龄段比重等；

2、原始数据“userdata.txt”的简单说明（感觉手机号是Base64方式编码的）：
  1）索引0：手机号（MSISDN）；
  2）索引11：使用日期（长整型的值）；
  3）索引12：使用时长；
  4）索引15：appid（对应appTab.txt里的appid，注意有的值前面补0了）；

3、数据“userdata_new.txt”是我处理后的数据：模拟手机号、appid、使用日期、使用时长；

In [148]:
app_tab = pd.read_csv('data/appTab.txt',encoding='gb18030',sep='|',names=['app_id','name','male','female','_24','25_30','31_35','36_40','40_'])
app_tab = app_tab.set_index('app_id')



In [149]:
user_data = pd.read_csv('data/userdata_new.txt',sep='|',names = ['tel','app_id','date','time'])

In [150]:
app_tab

,name,male,female,_24,25_30,31_35,36_40,40_
app_id,,,,,,,,
10001,QQ,0.001,0.001,0.000,0.200,0.300,0.200,0.300
10002,飞信,0.001,0.001,0.000,0.200,0.300,0.200,0.300
10003,MSN,0.001,0.001,0.000,0.200,0.300,0.200,0.300
10004,阿里旺旺,0.001,0.001,0.000,0.200,0.300,0.200,0.300
10005,微信,0.001,0.001,0.000,0.200,0.300,0.200,0.300
10006,陌陌,0.001,0.001,0.000,0.200,0.300,0.200,0.300
10007,米聊,0.001,0.001,0.000,0.200,0.300,0.200,0.300
10008,啪啪,0.001,0.001,0.000,0.200,0.300,0.200,0.300
10009,飞聊,0.001,0.001,0.000,0.200,0.300,0.200,0.300


In [151]:
user_data

,tel,app_id,date,time
0,13900000001,50057,1471020386250,70
1,13900000002,50057,1471017860712,75
2,13900000003,10005,1471018898111,161
3,13900000004,10005,1471020522545,107
4,13900000005,10005,1471020528891,20596
5,13900000005,10005,1471020110652,88080
6,13900000005,10005,1471020537933,148
7,13900000005,10005,1471020550412,7515
8,13900000005,10005,1471020538253,188
9,13900000005,10005,1471017990274,121


In [152]:
test_data = app_tab.loc[[10001],['male','female']]

In [153]:
test_data = np.array(test_data).flat

In [154]:
test_data

In [155]:
from pyecharts import Pie
def gender_rate(app_tab,app_id):
    gender_data = app_tab.loc[[app_id],['male','female']]
    gender_data = np.array(gender_data).flat
    attr = ['male','female']
    v1 = gender_data
    pie = Pie(get_name(app_id)+"男女比例")
    pie.add("", attr, v1, is_label_show=True)
    return pie

In [167]:
gender_rate(app_tab,10002)

In [157]:
from pyecharts import Pie
def age_distribution(app_tab,app_id):
    age_data = app_tab.loc[[app_id],['_24','25_30','31_35','36_40','40_']]
    age_data = np.array(age_data).flat
    attr = ['24岁以下','25-30岁','31-35岁','36-40岁','40岁以上']
    v1 = age_data
    pie = Pie(get_name(app_id)+"年龄分布")
    pie.add("", attr, v1, is_label_show=True)
    return pie

In [158]:
age_distribution(app_tab,10001)

In [159]:
def get_name(id):
    return app_tab.loc[id,'name']

In [160]:
# 平均用户留存时间
def avg_time(app_id):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-160-0a6a8059c019>, line 3)

In [164]:
user_data.fillna(0)
app_tab['avg_time'] = user_data.groupby('app_id')['time'].transform('mean')

In [165]:
app_tab

,name,male,female,_24,25_30,31_35,36_40,40_,avg_time
app_id,,,,,,,,,
10001,QQ,0.001,0.001,0.000,0.200,0.300,0.200,0.300,4273.580277
10002,飞信,0.001,0.001,0.000,0.200,0.300,0.200,0.300,4273.580277
10003,MSN,0.001,0.001,0.000,0.200,0.300,0.200,0.300,4160.802469
10004,阿里旺旺,0.001,0.001,0.000,0.200,0.300,0.200,0.300,2126.387133
10005,微信,0.001,0.001,0.000,0.200,0.300,0.200,0.300,6079.100000
10006,陌陌,0.001,0.001,0.000,0.200,0.300,0.200,0.300,2126.387133
10007,米聊,0.001,0.001,0.000,0.200,0.300,0.200,0.300,7318.458530
10008,啪啪,0.001,0.001,0.000,0.200,0.300,0.200,0.300,9095.086066
10009,飞聊,0.001,0.001,0.000,0.200,0.300,0.200,0.300,9095.086066


In [166]:
user_data.isnull().sum()

tel       0
app_id    0
date      0
time      0
dtype: int64